In [7]:
:dep itertools = "0.10.4"
:dep plotters = "0.3.4"
:dep rand = "0.8.5"
// Some imports and cargo deps
use std::{fs, collections::HashSet, iter::FromIterator, hash::Hash};
use itertools::Itertools;
use plotters::prelude::*;

In [ ]:
struct OpenReadingFrame {
    start: bool,
    stop: bool,
    reverse: bool
}

In [8]:
// Function to read dna string from file
fn get_dna_string(path: String) -> String {
    // read file with genome
    let raw_genome = fs::read_to_string(path).unwrap();
    // remove header line and replace newline by nothing
    raw_genome[(raw_genome.chars().position(|c| c == '\n').unwrap())..].replace("\n", "")
}

In [9]:
// create and auto transform histogram from data and save to path
fn create_histogram(path: String, data: Vec<isize>) {
    let unique_data = HashSet::<isize>::from_iter(data.clone().into_iter());
    let data_amount: Vec<isize> = unique_data.iter().map(|f| data.iter().filter(|v| **v == *f).count() as isize).collect();

    let drawing_area = SVGBackend::new(&path, (1920 * 2, 1080 * 2)).into_drawing_area();
    drawing_area.fill(&WHITE).unwrap();

    let mut chart = ChartBuilder::on(&drawing_area)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(50)
        .caption("Genome Length", ("sans-serif", 100.0))
        .build_cartesian_2d(0..unique_data.len() as isize, 0..*data_amount.iter().max().unwrap()).unwrap();

    chart
        .configure_mesh()
        .disable_x_mesh()
        .bold_line_style(&WHITE.mix(0.3))
        .y_desc("Amount")
        .x_desc("Length")
        .axis_desc_style(("sans-serif", 50))
        .draw()
        .unwrap();

    chart.draw_series(
        Histogram::vertical(&chart)
            .style(BLUE.filled())
            .data(data.iter().map(|x: &isize| (*x, 1)))
            .margin(0),
    ).unwrap();

    drawing_area.present().unwrap();
}

In [14]:
fn parse_orfs(dna: String, reverse: bool, triplets: triplet_dict) -> Vec<OpenReadingFrame> {
    let orfs = vec![];
    for offset in [0, 1, 2] {
        let mut last_start = -1;
        let mut found_start = false;
        for pos in (offset..dna.len()).step_by(3) {
            if pos + 2 < dna.len() {
                let current_triplet = &dna[pos..pos + 3];
                if current_triplet == start_codon && !found_start {
                    found_start = true;
                    last_start = pos as isize
                }
                if stop_codons.contains(&current_triplet) && found_start {
                    found_start = false;
                    orfs.push((last_start as usize + 1, pos as usize + 3, true));
                }
            }
        }
    }
    orfs
}

Error: cannot find value `start_codon` in this scope

Error: cannot find value `stop_codons` in this scope

In [10]:
// get DNA string
let dna = get_dna_string("genome.fna".to_string());
// let dna = "TTATGCATGCATAGATAA";

// codons
let start_codon = "ATG";
let stop_codons = ["TAG", "TAA", "TGA"];

// store found frames
let mut orfs = Vec::<(usize, usize, bool)>::new();

// check forward
for offset in [0, 1, 2] {
    let mut last_start = -1;
    let mut found_start = false;
    for pos in (offset..dna.len()).step_by(3) {
        if pos + 2 < dna.len() {
            let current_triplet = &dna[pos..pos + 3];
            if current_triplet == start_codon && !found_start {
                found_start = true;
                last_start = pos as isize
            }
            if stop_codons.contains(&current_triplet) && found_start {
                found_start = false;
                orfs.push((last_start as usize + 1, pos as usize + 3, true));
            }
        }
    }
}

// reverse and reverse compliment
let reversed_genome: String = dna.chars().map(|c| match c {
    'A' => 'T',
    'T' => 'A',
    'G' => 'C',
    'C' => 'G',
    _ => {panic!()},
}).rev().collect();

// check backwards
for offset in [0, 1, 2] {
    let mut last_start = -1;
    let mut found_start = false;
    for pos in (offset..reversed_genome.len()).step_by(3) {
        if pos + 2 < reversed_genome.len() {
            let current_triplet = &reversed_genome[pos..pos + 3];
            if current_triplet == start_codon && !found_start {
                found_start = true;
                last_start = pos as isize
            }
            if stop_codons.contains(&current_triplet) && found_start {
                found_start = false;
                orfs.push((reversed_genome.len() - pos - 2, reversed_genome.len() - last_start as usize, false));
            }
        }
    }
}

// print found orfs
// for i in &orfs {
    // println!("{} {} {}", i.0, i.1, ["-", "+"][if i.2 { 1 } else { 0 }])
// }
// println!("{}", orfs.len());

let data: Vec<isize> = orfs.iter().map(|o| (o.1 - o.0) as isize).sorted().collect();

create_histogram(String::from("histogram.svg"), data);